In [1]:
import numpy
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import torch.onnx

In [5]:
class Cough_Sense(torch.nn.Module):
    
    def __init__(self):
        super(Cough_Sense, self).__init__()
        self.input_layer = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=8, stride=3, padding=True)
        self.relu = nn.ReLU()
        self.sigm = nn.Sigmoid()
        self.cnn_16_16_5 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=5, stride=2, padding=True)
        self.cnn_16_32_3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=True)
        self.fci = nn.Linear(in_features=288, out_features=16)
        self.fco = nn.Linear(in_features=16, out_features=1)
        self.out = nn.Sigmoid()
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.cnn_16_16_5(x)
        x = self.relu(x)
        x = self.cnn_16_16_5(x)
        x = self.relu(x)
        x = self.cnn_16_32_3(x)
        x = self.relu(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fci(x)
        x = self.sigm(x)
        x = self.fco(x)
        x = self.out(x)
        
        return x
    

        
        

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model = Cough_Sense()
Model.to(device)
summary(Model, (8, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 41]           1,040
              ReLU-2               [-1, 16, 41]               0
            Conv1d-3               [-1, 16, 20]           1,296
              ReLU-4               [-1, 16, 20]               0
            Conv1d-5                [-1, 16, 9]           1,296
              ReLU-6                [-1, 16, 9]               0
            Conv1d-7                [-1, 32, 9]           1,568
              ReLU-8                [-1, 32, 9]               0
            Linear-9                   [-1, 16]           4,624
          Sigmoid-10                   [-1, 16]               0
           Linear-11                    [-1, 1]              17
          Sigmoid-12                    [-1, 1]               0
Total params: 9,841
Trainable params: 9,841
Non-trainable params: 0
-----------------------------------

In [20]:
Model.to(device)
Model(torch.rand(1,8,128).to(device))

tensor([[0.6546]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [7]:
D = torch.rand(10,8,128).to(device)
O = Model(D)

loss = nn.BCELoss()
o = loss(O, torch.rand(10,1).to(device))
o.backward()

In [9]:
torch.onnx.export(Model.to(torch.device('cpu')), torch.rand(1,8,128), "test.onnx", export_params=True, opset_version=10, do_constant_folding=True, input_names=['input_layer'], output_names=['out'])

In [11]:
import onnx
M = onnx.load('test.onnx')

In [13]:
onnx.checker.check_model(M)

In [14]:
onnx.helper.printable_graph(M.graph)

'graph torch-jit-export (\n  %input_layer[FLOAT, 1x8x128]\n) initializers (\n  %cnn_16_16_5.bias[FLOAT, 16]\n  %cnn_16_16_5.weight[FLOAT, 16x16x5]\n  %cnn_16_32_3.bias[FLOAT, 32]\n  %cnn_16_32_3.weight[FLOAT, 32x16x3]\n  %fci.bias[FLOAT, 16]\n  %fci.weight[FLOAT, 16x288]\n  %fco.bias[FLOAT, 1]\n  %fco.weight[FLOAT, 1x16]\n  %input_layer.bias[FLOAT, 16]\n  %input_layer.weight[FLOAT, 16x8x8]\n) {\n  %11 = Conv[dilations = [1], group = 1, kernel_shape = [8], pads = [1, 1], strides = [3]](%input_layer, %input_layer.weight, %input_layer.bias)\n  %12 = Relu(%11)\n  %13 = Conv[dilations = [1], group = 1, kernel_shape = [5], pads = [1, 1], strides = [2]](%12, %cnn_16_16_5.weight, %cnn_16_16_5.bias)\n  %14 = Relu(%13)\n  %15 = Conv[dilations = [1], group = 1, kernel_shape = [5], pads = [1, 1], strides = [2]](%14, %cnn_16_16_5.weight, %cnn_16_16_5.bias)\n  %16 = Relu(%15)\n  %17 = Conv[dilations = [1], group = 1, kernel_shape = [3], pads = [1, 1], strides = [1]](%16, %cnn_16_32_3.weight, %cnn_16

In [15]:
M.graph

node {
  input: "input_layer"
  input: "input_layer.weight"
  input: "input_layer.bias"
  output: "11"
  name: "Conv_0"
  op_type: "Conv"
  attribute {
    name: "dilations"
    ints: 1
    type: INTS
  }
  attribute {
    name: "group"
    i: 1
    type: INT
  }
  attribute {
    name: "kernel_shape"
    ints: 8
    type: INTS
  }
  attribute {
    name: "pads"
    ints: 1
    ints: 1
    type: INTS
  }
  attribute {
    name: "strides"
    ints: 3
    type: INTS
  }
}
node {
  input: "11"
  output: "12"
  name: "Relu_1"
  op_type: "Relu"
}
node {
  input: "12"
  input: "cnn_16_16_5.weight"
  input: "cnn_16_16_5.bias"
  output: "13"
  name: "Conv_2"
  op_type: "Conv"
  attribute {
    name: "dilations"
    ints: 1
    type: INTS
  }
  attribute {
    name: "group"
    i: 1
    type: INT
  }
  attribute {
    name: "kernel_shape"
    ints: 5
    type: INTS
  }
  attribute {
    name: "pads"
    ints: 1
    ints: 1
    type: INTS
  }
  attribute {
    name: "strides"
    ints: 2
    typ